In [2]:
%pip install kaggle

  Using cached kaggle-1.5.12-py3-none-any.whl
  Using cached python_slugify-6.1.2-py2.py3-none-any.whl (9.4 kB)
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 kB 909.4 kB/s eta 0:00:00a 0:00:01
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 667.9 kB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=375ade65bd0216c20e250cdbc46e7f2fb3e9c8efec9143ed018bc2baa62b094a
  Stored in directory: /home/lion/.cache/pip/wheels/81/9c/6c/d5200fcf351ffa39cbe09911e99703283624cd037df58070d9
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install nltk

  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.0/764.0 kB 701.0 kB/s eta 0:00:0000:0100:01
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 2.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 1.8 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

# Preprocessing
We're going to preprocess the original dataset from Kaggle to reduce its size and only work on meaningful data for our analysis: 
1. unzip the provided file to work on individual CSV files;
2. filter only English written tweets to build a coherent language base;
3. remove useless columns such as the account description or the number of retweets;
4. possibly remove some stop-words.

In [ ]:
import time

start_time = time.time()

## Code imports and globals

In [ ]:
import zipfile
import os
import pyspark
import regex
import shutil
import gzip
import nltk
from nltk.tokenize import word_tokenize
import string
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.types import Row
from pyspark.sql.functions import concat_ws
import multiprocessing
import pandas
from typing import List, Dict
from datetime import datetime
from nltk.corpus import stopwords
import json


TOP_HASHTAGS = ['ukraine', 'russia', 'standwithukraine', 'putin', 'russian', 'mariupol', 'ukrainerussiawar', 'nato', 'tigray', 'ukrainewar', 'kyiv', 'ukrainian', 'stoprussia', 'news', 'armukrainenow', 'russiaukrainewar', 'usa', 'stopputin', 'kharkiv', 'slavaukraini', 'bucha', 'ukrainerussianwar', 'biden', 'ukraineunderattack', 'business', 'war', 'russianukrainianwar', 'stopputinnow', 'azovstal', 'eu', 'ukraine️', 'kherson', 'anonymous', 'standwithukraine️', 'russianwarcrimes', 'china', 'us', 'tigraygenocide', 'russians', 'zelensky', 'russianarmy', 'europe', 'endtigraysiege', 'nft', 'russiaukraine', 'belarus', 'breaking', 'ukrainerussia', 'ethiopia', 'poland', 'buchamassacre', 'ukraineunderattaсk', 'warcrimes', 'freeukraine', 'germany', 'zelenskyy', 'safeairliftukraine', 'donetsk', 'donbass', 'putinisawarcriminal', 'stopwar', 'ukrainians', 'moscow', 'kiev', 'ukraineinvasion', 'stoprussianaggression', 'putinwarcriminal', 'finland', 'trump', 'azov', 'uk', 'russiainvadedukraine', 'stopthewar', 'oprussia', 'canada', 'putinswar', 'russiaukraineconflict', 'putinwarcrimes', 'nfts', 'warinukraine', 'syria', 'india', 'peace', 'sweden', 'tigraycantwait', 'savemariupol', 'standupforukraine', 'humanitycomesfirst', 'helpukraine', 'supportukraine', 'odessa', 'ausgov', 'severodonetsk', 'eurovision', 'luhansk', 'nftcommunity', 'irpin', 'crypto', 'bitcoin', 'chernihiv']


def update_top_hashtags():
    global TOP_HASHTAGS_INDEX, TOP_HASHTAGS_REVERSE_INDEX
    zipped_hashtags = list(zip(TOP_HASHTAGS, range(len(TOP_HASHTAGS))))
    TOP_HASHTAGS_INDEX = {key: value for key, value in zipped_hashtags}
    TOP_HASHTAGS_REVERSE_INDEX = {value: key for key, value in zipped_hashtags}

TOP_HASHTAGS_INDEX = dict()
TOP_HASHTAGS_REVERSE_INDEX = dict()
update_top_hashtags()



## Configuration

In [ ]:
DO_PREPROCESS = True
FIND_MOST_COMMON = False

TOP_HASHTAGS_INDEX_FILENAME = "top_hashtags_index.json"
TOP_HASHTAGS_REVERSE_INDEX_FILENAME = "top_hashtags_reverse_index.json"

KAGGLE_DATASET = "ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip"
KAGGLE_DATASET_DIRECTORY = os.path.join("out", os.path.splitext(KAGGLE_DATASET)[0])
WORKERS_CORES = multiprocessing.cpu_count()
FILTER_LANGUAGE = "en"

## Stopwords list retrieval

In [ ]:
def update_nltk() -> None:
    nltk.download('stopwords')
    nltk.download('punkt')

## Dataset extraction
Unzip the kaggle dataset. If an extracted version of that dataset is already present, only extract new files. After that, extract .gzip files into .csv

In [ ]:
def dataset_extraction(archive: str, output_directory: str) -> List[str]:
    new_files = list()
    with zipfile.ZipFile(archive, "r") as zip_ref:

        #return [os.path.join(output_directory, csv_name) for csv_name, _ in [os.path.splitext(g_name) for g_name in zip_ref.namelist()[:2]]]

        if not os.path.isdir(output_directory):
            zip_ref.extractall(output_directory)
            #new_files = zip_ref.namelist()
        # else:
        for gzip_name in zip_ref.namelist():
            csv_name, extension = os.path.splitext(gzip_name)
            if os.path.isfile(os.path.join(output_directory, csv_name)):
                continue
            if not os.path.isfile(os.path.join(output_directory, gzip_name)):
                zip_ref.extract(gzip_name, path=output_directory)
            csv_path = os.path.join(output_directory, csv_name)
            with gzip.open(os.path.join(output_directory, gzip_name), 'r') as gzip_file, open(csv_path, 'wb') as csv_file:
                shutil.copyfileobj(gzip_file, csv_file)
            new_files.append(csv_path)
    return new_files

## Spark initialization

In [ ]:
def init_spark() -> pyspark.sql.SparkSession:
    print(f"Available CPU cores/workers: {WORKERS_CORES}")
    print("Initializing spark...", end=' ', flush=True)
    spark = (
        pyspark.sql.SparkSession.builder
        .master(f"local[{WORKERS_CORES}]")
        .appName("Sparkiodi")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("OFF")
    print("Spark initialized")
    return spark

## Dataset reduction
Read all the CSVs in the specified path. "path" can be a list of files, a folder containig multiple files, or a regex matching multiple files.

In [ ]:
def read_dataframe(path: List[str], spark, header: bool = True, language: str = None) -> pyspark.sql.DataFrame:
    if header:
        starting_df = (
            spark
            .read
            .option("header", True)
            .option("multiLine", True)
            .csv(path)
        )
    else:
        starting_df = (
            spark
            .read
            .option("header", False)
            .option("multiLine", True)
            .csv(path)
        )

    if language:
        broadcast_language = spark.sparkContext.broadcast(FILTER_LANGUAGE)
        starting_df = starting_df.where(
            starting_df.language == broadcast_language.value)

    return starting_df.select("text")

### Entities removal and hashtags extraction

#### Regex
Match hashtags, URLs, user-mentions, and punctuation, to eventually remove them from the tweet's text

In [ ]:
hashtag_regex_str = r"(?:\#+)([\w_]+)"  # hashtags
regex_str = [
    (r'(?:@[\w_]+)', ''),  # @-mentions
    (r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', ''), # URLs
    (r'[^\w\s]', ' '),  # punctuation
    (r'\s+', ' ')  # whitespaces
]

hashtag_regex = regex.compile(hashtag_regex_str)
regex = [(hashtag_regex, '')] + [(regex.compile(compiled[0]), compiled[1])
                                 for compiled in regex_str]   # Keep hashtag_regex as the first applied regex

#### Text cleaning
Use the the regexes to filter out from the text anything that's not a meaningful word

In [ ]:
def clean_text(text: str) -> List[str]:
    text = text.lower()

    for reg in regex:
        text = reg[0].sub(reg[1], text)

    text_list = set([
        word.rstrip() for word in word_tokenize(text)
        if word not in string.punctuation
    ])

    return list(text_list)

In [ ]:
def padding_hashtags(hashtags: List[str], indexes: Dict[str, int], compress:Bool=False):
    """ Utility function to create an indicator vector for the 'hashtags' list over the 'indexes' parameter
    
    Args:
        hashtags (List[str]): list of hashtags to be mapped into an indicator vector
        indexes (Dict[str, int]): mapping from an hashtag to its index position
        compress (Bool): decides wether to store the whole indicator vector (False) or only the non-zero indices (True)
    
    Returns:
        List[int]: a list of integer with the most common hashtags for the initial list
    """
    if compress:
        pads = list()
        for tag in hashtags:
          if tag in indexes:
              pads.append(indexes[tag])
        pads.sort()
    else:
        pads = [0 for _ in range(len(indexes))]
        for tag in hashtags:
            if tag in indexes:
                pads[indexes[tag]] = 1
    return pads


def clean_dataframe(df:pyspark.sql.DataFrame, spark:pyspark.sql.SparkSession)-> pyspark.sql.DataFrame:

    # StopWordsRemover is a pyspark utility to remove words from a dataset's column
    remover = StopWordsRemover(stopWords=stopwords.words('english'))
    remover.setInputCol("tweet")
    remover.setOutputCol("filtered_tweet")

    # From every row extract the "cleaned" text and a list of its hashtags
    tweets_hashtags_rdd = (
        df.rdd
        .map(lambda row: (clean_text(row.text), [ht.lower() for ht in hashtag_regex.findall(row.text)]))
    )

    if FIND_MOST_COMMON:
        global TOP_HASHTAGS
        TOP_HASHTAGS = (
            tweets_hashtags_rdd
            .flatMap(lambda row: row[1])
            .map(lambda row: (row, 1))
            .reduceByKey(lambda x, y: x + y)
            .map(lambda row: (row[1], row[0]))
            .sortByKey(ascending=False)
            .map(lambda row: row[1])
            .take(100)
        )
        update_top_hashtags()

    broadcast_top_hashtags_index = spark.sparkContext.broadcast(TOP_HASHTAGS_INDEX)

    # Map the second column (containing the hashtags) into its indicator vector
    df = (
        tweets_hashtags_rdd
        .map(lambda row: (row[0], padding_hashtags(row[1], broadcast_top_hashtags_index.value, True)))
        .toDF(["tweet", "hashtags"])
    )
    # Remove the stop words and concatenate each column into a whitespace separated list
    df = (
        remover
        .transform(df)
        .select("filtered_tweet", "hashtags")
        .withColumn("filtered_tweet", concat_ws(" ", "filtered_tweet"))
        .withColumn("hashtags", concat_ws(" ", "hashtags"))
    )
    # Filter out eventual tweets without text
    df = df.where((df.filtered_tweet != ""))

    return df

## Preprocessed dataset storing

In [ ]:
def write_preprocessed(df:pyspark.sql.DataFrame, output_directory: str) -> str:
    print("Writing top hashtags...")
    with open(TOP_HASHTAGS_INDEX_FILENAME, "w") as fp:
        json.dump(TOP_HASHTAGS_INDEX, fp)
    with open(TOP_HASHTAGS_REVERSE_INDEX_FILENAME, "w") as fp:
        json.dump(TOP_HASHTAGS_REVERSE_INDEX, fp)
    print("Writing all the CSVs...")
    (df
        # .coalesce(WORKERS_CORES)
        .write
        .mode("overwrite")
        .option("header", False)
        .csv(output_directory)
     )

    # spark writes one file per task along with its CRC, so we delete all the non-CSV files we don't require
    for dirpath, dirnames, filenames in os.walk(output_directory):
        for filename in filenames:
            name, extension = os.path.splitext(filename)
            if extension == ".csv":
                continue
            filepath = os.path.join(dirpath, filename)
            try:
                os.remove(filepath)
            except:
                pass

In [ ]:
def compress(path: str) -> str:
    print("Creating archive directory...")
    if os.path.isfile(path):
        name, ext = os.path.splitext(path)
        output_archive = f"{name}.zip"
        with zipfile.ZipFile(output_archive, "w", compression=zipfile.ZIP_DEFLATED) as zf:
            zf.write(path, os.path.basename(path))
    else:
        output_archive = shutil.make_archive(path, 'zip', path)
    return output_archive

In [ ]:
def create_single_csv(folders: List[str], overwrite: bool = False) -> str:
    single_filename = 'out/dataset.csv'
    filemode = 'wb' if overwrite else 'ab'
    with open(single_filename, filemode) as output_file:
        for directory in folders:
            for dirpath, dirnames, filenames in os.walk(directory):
                for filename in filenames:
                    name, extension = os.path.splitext(filename)
                    if extension != ".csv":
                        continue
                    filepath = os.path.join(dirpath, filename)
                    with open(filepath, 'rb') as csv:
                        shutil.copyfileobj(csv, output_file)
    return single_filename

## Start preprocessing

In [ ]:
def preprocess() -> None:
    new_files = dataset_extraction(KAGGLE_DATASET, KAGGLE_DATASET_DIRECTORY)
    if not len(new_files):
        print("No new files to process")
        return

    update_nltk()
    if not ('spark' in locals() or 'spark' in globals()):
        spark = init_spark()
    df = read_dataframe(new_files, spark, language=FILTER_LANGUAGE)
    df = clean_dataframe(df, spark)
    output_directory = f"out/preprocessed_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
    write_preprocessed(df, output_directory)
    csv = create_single_csv([output_directory])
    compress(csv)

if DO_PREPROCESS:
    preprocess()

In [ ]:
end_time = time.time()
print(f"Start time: {start_time}")
print(f"End time: {end_time}")
print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(end_time - start_time))}")

In [ ]:
!head out/dataset.csv